In [65]:
import pandas as pd
import numpy as np
import zipfile
import sys
sys.path.append('../')
from helper import spkly
from helper import db
import datetime
import math


In [63]:
zip = zipfile.ZipFile('../data/LTTMP_1980_2019.zip')

df = pd.DataFrame()
for i in [i for i in zip.namelist() if 'csv' in i]:
    print(i)
    datafile = pd.read_csv(zip.open(i), encoding='cp437')
    datafile.columns = ['station_id', 'latitude', 'longitude', 'measure_time', 'depth', 'temperature', 'salinity']
    datafile['measure_time'] = pd.to_datetime(datafile['measure_time'])
    df = pd.concat([df, datafile])

Data/01966_SHC.csv
Data/01970_SHC.csv
Data/02330_SHC.csv
Data/02780_SHC.csv
Data/02985_SHC.csv
Data/03057_SHC.csv
Data/03100_SHC.csv
Data/03246_SHC.csv
Data/03248_SHC.csv
Data/03250_SHC.csv
Data/03300_SHC.csv
Data/03335_SHC.csv
Data/03353_SHC.csv
Data/03360_SHC.csv
Data/03365_SHC.csv
Data/03460_SHC.csv
Data/15930_SHC.csv
Data/BANC_AMERICAINS.csv
Data/BBE_MT.csv
Data/BBY_MT.csv
Data/BCO_MT.csv
Data/BIC_MT.csv
Data/BOR_MT.csv
Data/BSB_MT.csv
Data/COURANT_GASPE.csv
Data/GRI_MT.csv
Data/GYRE_ANTICOSTI.csv
Data/HSP_MT.csv
Data/IRW_MT.csv
Data/ISH_MT.csv
Data/LAR_MT.csv
Data/MLO_MT.csv
Data/NAT_MT.csv
Data/OLD_HARRY.csv
Data/PER_MT.csv
Data/POM_MT.csv
Data/RAT_MT.csv
Data/RIKI.csv
Data/SHEDIAC.csv
Data/SIL_MT.csv
Data/TAB_MT.csv
Data/TAD_MT.csv


In [73]:
df.dropna(inplace=True)
subset = df[df['measure_time']>=datetime.datetime(2009,1,1)].copy()
subset['measure_time'] = subset['measure_time'].dt.date

bin_max = (math.ceil(subset['depth'].max() / 100) + 1) * 100
cut_bins = [i for i in range(0, bin_max, 100)]
subset['depth_bin'] = pd.cut(subset['depth'], cut_bins, right=False)
subset['depth_bin'] = subset['depth_bin'].astype('str')

subset = subset.groupby(['station_id', 'latitude', 'longitude', 'measure_time', 'depth_bin'])[['temperature', 'salinity']].mean().reset_index()
subset['data_source'] = 'DFO Quebec'

RENAME_COLUMNS = {'measure_time': 'data_date', 'station_id': 'floatid', 'depth_bin': 'depth'}
subset.rename(columns=RENAME_COLUMNS, inplace=True)

In [76]:
db.insert_table(table_name='ocean_data', df=subset)

In [47]:
subset.shape

,station,latitude,longitude,date,depth_bin,temperature,salinity,data_source
0,166930,47.3786,-61.8566,2009-01-01 00:00:00,"[0, 100)",-0.75,29.27,DFO Quebec
1,166930,47.3786,-61.8566,2009-01-01 01:00:00,"[0, 100)",-0.77,29.28,DFO Quebec
2,166930,47.3786,-61.8566,2009-01-01 02:00:00,"[0, 100)",-0.79,29.28,DFO Quebec
3,166930,47.3786,-61.8566,2009-01-01 03:00:00,"[0, 100)",-0.80,29.30,DFO Quebec
4,166930,47.3786,-61.8566,2009-01-01 04:00:00,"[0, 100)",-0.80,29.30,DFO Quebec


In [15]:
import folium

import folium
m = folium.Map(location=[50, -60], zoom_start=4.5, tiles='Stamen Terrain')

for flt in temp_table.index[:10]:
    folium.Marker(location=[temp_table["Latitude"].iloc[flt], temp_table["Longitude"].iloc[flt]]).add_to(m)
m

""
